# Understanding Plonk (V): Polynomial Commitment

## What is a Polynomial Commitment?

A commitment is a way to "lock" a message and obtain a commitment value. This value is referred to as the "commitment" of an object:

$$
c = \text{commit}(x)
$$

The commitment has two key properties: Hiding and Binding.
- Hiding： $c$ does not reveal any information about $x$；
- Binding：It is computationally hard to find a different $x', x'\neq x$，such that $c=\text{commit}(x')$.

The simplest form of commitment is a cryptographic hash function, such as SHA-256 or Keccak, which provides cryptographic security. Other commitment schemes include Pedersen commitments.

As the name suggests, a polynomial commitment is a commitment scheme applied to polynomials. If we express a polynomial as:

$$
f(X) = a_0 + a_1X + a_2X^2 + \cdots + a_nX^n
$$

we can uniquely identify the polynomial $f(X)$ using its coefficient vector:

$$
(a_0, a_1, a_2,\ldots, a_n)
$$

How can we commit to a polynomial? One straightforward approach is to apply a hash function to the coefficient vector, obtaining a unique binding value:

$$
C_1 = \textrm{SHA256}(a_0\parallel a_1 \parallel a_2 \parallel \cdots \parallel a_n)
$$

Alternatively, we can use a Pedersen commitment, which computes an elliptic curve (ECC) point using a set of randomly chosen bases:

$$
C_2 = a_0 G_0 + a_1  G_1 + \cdots + a_n G_n
$$

Once the prover commits to a polynomial, the verifier may want to evaluate the committed polynomial at a specific point and have the prover provide proof of the evaluation's correctness. Suppose $C=\text{Commit}(f(X))$, the verifier can ask the prover for the value of the polynomial at $X=\zeta$. The prover will return not only the result $f(\zeta)=y$ but also a proof $\pi$ that verifies that $f(X)$ is indeed the polynomial committed to by $C$.

This "evaluation with proof" feature of polynomial commitments is extremely useful. It enables a lightweight form of verifiable computation, where a verifier delegates the polynomial computation to a remote machine (the prover) and then verifies the correctness of the result $y$ with less computational effort than directly computing $f(X)$. Additionally, polynomial commitments can be used to prove properties of secret data (held by the prover) without revealing the data itself. The prover can convince the verifier that their data satisfies a certain polynomial equation while keeping the data private.

Although verifiable computation through polynomial commitments is limited to polynomial operations rather than general computations, general computations can often be transformed into polynomial computations, making polynomial commitments a fundamental tool for verifiable computing.

Using the Pedersen commitment scheme from $C_2$​, we can leverage protocols such as Bulletproof-IPA to implement evaluation proofs, leading to another polynomial commitment scheme. Other well-known polynomial commitment schemes include KZG10, FRI, Dark, and Dory.

## KZG Polynomial Commitment Scheme

Compared to the random basis vectors used in Pedersen commitments, the KZG10 polynomial commitment requires a set of basis vectors with internal algebraic structure instead.

$$
(G_0, G_1, G_2, \ldots, G_{d-1}, H_0, H_1) = (G, \alpha G, \alpha^2G, \ldots, \alpha^{d-1}G, H, \alpha H)
$$

Here, $\alpha$ is a random value provided by a trusted third party, also known as the trapdoor. This value must be permanently deleted after the trusted setup process. Neither the verifier nor the prover should know $\alpha$.

In this setup:
- $G \in \mathbb{G}_1$​, and $H \in \mathbb{G}_2$,
- There exists a bilinear pairing $e: \mathbb{G}_1 \times \mathbb{G}_2 \rightarrow \mathbb{G}_T​$.

To commit to a polynomial $f(X)$ using KZG1, we commit to its coefficient vector:

$$
\begin{split}
C_{f(X)} &= a_0 G_0 + a_1  G_1 + \cdots + a_{n-1} G_{n-1} \\
 & = a_0  G + a_1 \alpha G + \cdots + a_{n-1}\alpha^{n-1} G\\
 & = f(\alpha) G
\end{split}
$$

For bilinear groups, we use the notation introduced by Groth: $[1]_1\triangleq G$， $[1]_2\triangleq H$ to represent the generators of the two groups. With this notation, the KZG10 system parameters (also known as the Structured Reference String, SRS) can be expressed as:

$$
\mathsf{srs}=([1]_1,[\chi]_1,[\chi^2]_1,[\chi^3]_1,\ldots,[\chi^{n-1}]_1,[1]_2,[\chi]_2)
$$

And the polynomial commitment is given by: $C_{f(X)}=[f(\alpha)]_1$。

Below, we use the polynomial obtained in the previous section as an example to generate the commitment:

$$
f(X) = 40X^6 + 73X^5 + 32X^4 + 61X^2 + 28X + 69
$$


In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install py_ecc curve typing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 48.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 56.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.5/254.5 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.0/385.0 kB 321.5 kB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 6.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/

In [2]:
import py_ecc.bn128 as b
from typing import NewType

G1Point = NewType("G1Point", tuple[b.FQ, b.FQ])
G2Point = NewType("G2Point", tuple[b.FQ2, b.FQ2])

class Setup():
    #   ([1]₁, [x]₁, ..., [x^{d-1}]₁)
    # = ( G,    xG,  ...,  x^{d-1}G ), where G is a generator of G_1
    powers_of_x: list[G1Point]
    # [x]₂ = xH, where H is a generator of G_2
    X2: G2Point

    # tau: a random number whatever you choose
    def __init__(self, powers: int, tau: int):
        print("Start to generate structured reference string")

        # Initialize powers_of_x with 0 values
        self.powers_of_x = [0] * powers
        # powers_of_x[0] =  b.G1 * tau**0 = b.G1
        # powers_of_x[1] =  b.G1 * tau**1 = powers_of_x[0] * tau
        # powers_of_x[2] =  b.G1 * tau**2 = powers_of_x[1] * tau
        # ...
        # powers_of_x[i] =  b.G1 * tau**i = powers_of_x[i - 1] * tau
        self.powers_of_x[0] = b.G1

        for i in range(powers):
            if i > 0:
                self.powers_of_x[i] = b.multiply(self.powers_of_x[i - 1], tau)

        print("Generated G1 side, X^1 point: {}".format(self.powers_of_x[1]))

        self.X2 = b.multiply(b.G2, tau)
        print("Generated G2 side, X^1 point: {}".format(self.X2))

        print("Finished to generate structured reference string")

    # Encodes the KZG commitment that evaluates to the given values in the group
    def commit(self, coeffs) -> G1Point:
        commitment = b.Z1
        for index, coeff in enumerate(coeffs):
            ec_point = b.multiply(self.powers_of_x[index], coeff % b.curve_order)
            commitment = b.add(commitment, ec_point)

        return commitment

In [3]:
# random number
tau = 74

# powers number should be larger than the degree of polynomial you want to commit
powers = 8

# do the setup
setup = Setup(powers, tau)

# combined polynomial combined_poly from last chapter:  40x^6 + 73x^5 + 32x^4 + 61x^2 + 28x + 69
f_coeffs = [69, 28, 61, 0, 32, 73, 40]

# commit
C_f = setup.commit(f_coeffs)
print("C_f: ", C_f)

assert C_f == (5944924129024846722741625252433644255755361576692942464620418422112209381657, 8343652485787411819127825636992247747330773247827507108642802455713543154102)

Start to generate structured reference string
Generated G1 side, X^1 point: (19000714569087058254079111722938672430276630300266312265196309930792761914189, 9925954159276340969458888695294901436812701424573926030176685839770908267539)
Generated G2 side, X^1 point: ((1143807547817528759872448485706976526436907032146971695798891422984531866726, 12471147282413329518352649295925560204177370150234424204094147917137349478087), (3714552804415258437881936892866262499095335352823341615390152505710682815003, 1775681321792385927298379567244671335181813357654154576426547499429277592831))
Finished to generate structured reference string
C_f:  (5944924129024846722741625252433644255755361576692942464620418422112209381657, 8343652485787411819127825636992247747330773247827507108642802455713543154102)


Below, we construct an Open proof for $f(\zeta) = y$. According to the polynomial remainder theorem, we derive the following equation:

\begin{align*}
\frac{f(X) - y}{X - \zeta} &= q(X) \\
\implies f(X) - y &= q(X)\cdot (X-\zeta)
\end{align*}

Since $f(\zeta) = y$, the polynomial $f(X) - y$ has a root at $\zeta$. This means $(X - \zeta)$ is a factor of $f(X) - y$; in other words, there must exist a quotient polynomial $q(X)$ satisfying the equation above.

The Prover can then provide a commitment to the quotient polynomial $q(X)$, denoted as $[q(\alpha)]_1$​. The Verifier checks whether $[q(\alpha)]_1$ satisfies the divisibility condition. If $f(\zeta) \neq y$, then $g(X)$ cannot be divided by $(X−\zeta)$, and the commitment provided by the Prover will fail the divisibility check:

$$
(f(X)-y)\cdot 1 \overset{?}{=} q(X) \cdot (X-\zeta)
$$

The commitment $[f(\alpha)]_1$​ is an element in the group $\mathbb{G}_1$​. Using the homomorphic properties of commitments and the bilinear pairing property $e\in \mathbb{G}_1\times\mathbb{G}_2\to \mathbb{G}_T$​, the Verifier can check the divisibility condition in $\mathbb{G}_T$:

$$
e([f(\alpha)]_1 - y[1]_1, [1]_2) \overset{?}{=} e([q(\alpha)]_1, [\alpha]_2 - \zeta [1]_2)
$$

To reduce the computational cost for the Verifier when operating in $\mathbb{G}_2$​, the verification equation can be rewritten as:

$$
f(X) + \zeta\cdot q(X) - y =  q(X)\cdot X
$$

$$
e([f(\alpha)]_1 + \zeta\cdot [q(\alpha)]_1 -y\cdot[1]_1,\ [1]_2)\overset{?}{=} e([q(\alpha)]_1,\  [\alpha]_2)
$$


Continuing with the following polynomial as an example:
$$
f(X) = 40X^6 + 73X^5 + 32X^4 + 61X^2 + 28X + 69
$$

For the sake of illustration, pick the evaluation point $X=\zeta=1$，and compute $f(\zeta)=y=303$.

The quotient polynomial is:
$$q(X)=\frac{f(X)-y} {X-\zeta} = 40X^5 + 113X^4 + 145X^3 + 145X^2 + 206X + 234$$

Finally, we need to verify:

$$
e(C_{f(X)} + \zeta\cdot C_{q(X)} -y\cdot[1]_1,\ [1]_2)\overset{?}{=} e(C_{q(X)},\  [\alpha]_2)
$$

See below for the code implementation:

In [5]:
# random number
tau = 74

# powers number should be larger than the degree of polynomial you want to commit
powers = 8

# do the setup
setup = Setup(powers, tau)

# 40x^6 + 73x^5 + 32x^4 + 61x^2 + 28x + 69
f_coeffs = [69, 28, 61, 0, 32, 73, 40]

# random number
zeta = 1
# y = f(zeta), which is f(X) evaluate at X = zeta
y = 303

# q(X) = f(X)/(X-zeta) = 40X^5 + 113X^4 + 145X^3 + 145X^2 + 206X + 234
quot_coeffs = [234, 206, 145, 145, 113, 40]

C_f = setup.commit(f_coeffs)
C_q = setup.commit(quot_coeffs)
print("C_f: ", C_f)
print("C_q: ", C_q)

### do the linear combination for C_f + zeta * C_q - y * [1] ###

# zeta * C_q
C_linc_1 = b.multiply(C_q, zeta)
print("C_linc_1: ", C_linc_1)

# - y * [1]
assert -y % b.curve_order == 21888242871839275222246405745257275088548364400416034343698204186575808495314
assert b.G1 == setup.powers_of_x[0]
assert b.multiply(b.G2, tau) == setup.X2

C_linc_2 = b.multiply(b.G1, (-y) % b.curve_order)
assert C_linc_2 == (18788921900215882028576331803050066650912825348279745351731017781865893991129, 1073865593169846015410867644238358850025750640620148412555516903827118500108), "C_linc_2 not currect"
print("C_linc_2: ", C_linc_2)
assert b.multiply(C_f, 1) == C_f, "C_f should equal"

# final combination
C_linc = b.add(b.add(C_f, C_linc_1), C_linc_2)
print("C_linc: ", C_linc)
assert C_linc == (6301486514557274509058238130830016809489505352124366227551955002407651000262, 13048730005043404879416281954412709276285759583175910127109270903170794734865), "C_linc not correct"

assert b.pairing(b.G2, C_linc) == b.pairing(setup.X2, C_q), "Pairing not equal"

# no error, print success msg
print("Pairing check is successful!")

Start to generate structured reference string
Generated G1 side, X^1 point: (19000714569087058254079111722938672430276630300266312265196309930792761914189, 9925954159276340969458888695294901436812701424573926030176685839770908267539)
Generated G2 side, X^1 point: ((1143807547817528759872448485706976526436907032146971695798891422984531866726, 12471147282413329518352649295925560204177370150234424204094147917137349478087), (3714552804415258437881936892866262499095335352823341615390152505710682815003, 1775681321792385927298379567244671335181813357654154576426547499429277592831))
Finished to generate structured reference string
C_f:  (5944924129024846722741625252433644255755361576692942464620418422112209381657, 8343652485787411819127825636992247747330773247827507108642802455713543154102)
C_q:  (11740539305859663668512843267191890549789440068194701996652191809100675186622, 7923407303438355406747965479994761895007274674134214153265773933467813023745)
C_linc_1:  (117405393058596636685128432671